In [ ]:
import torch
import spacy
from myselfcheckgpt.modeling_selfcheck import SelfCheckNgram
import numpy as np
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
selfcheck_ngram = SelfCheckNgram(n=1) # n=1 means Unigram, n=2 means Bigram, etc.
nlp = spacy.load("en_core_web_sm")

input_file_name = "./input/en.jsonl"

/home/srongaa/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/srongaa/llm/selfcheckgpt/myselfcheckgpt/modeling_mqag.py:63: SyntaxWarning: invalid escape sequence '\S'
  distractors = re.sub("<extra\S+>", g2_tokenizer.sep_token, distractors)
/home/srongaa/llm/selfcheckgpt/myselfcheckgpt/modeling_mqag.py:130: SyntaxWarning: invalid escape sequence '\S'
  distractors = re.sub("<extra\S+>", g2_tokenizer.sep_token, distractors)
/home/srongaa/llm/selfcheckgpt/myselfcheckgpt/modeling_mqag.py:194: SyntaxWarning: invalid escape sequence '\S'
  distractors = re.sub("<extra\S+>", g2_tokenizer.sep_token, distractors)


SelfCheck-1gram initialized


In [ ]:
import json

def load_file_jsonl(file :str):
    data = []
    try:
        with open(file, mode='r', encoding='utf-8') as file:
            for line in file:
                # Parse each line as a JSON object
                json_obj = json.loads(line)
                data.append(json_obj)
    except FileNotFoundError:
        print(f"Error: File not found at {file}")
    except json.JSONDecodeError as e:
        print(f"Error: Invalid JSON format. {e}")
    except Exception as e:
        print(f"Error: {e}")

    return data

data = load_file_jsonl(input_file_name)
print(data[0]['model_input'])
print(data[0]['model_output_text'])
model_name = data[0]['model_id']

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True).to(device)

In [4]:
def ask(message_to_ask, model):
    #question here
    message = message_to_ask

    config = dict(top_k=50, top_p=0.90, temperature=0.1)
    prompt = message + '\n'
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(
    **inputs,
    max_new_tokens=512,
    num_return_sequences=1,
    no_repeat_ngram_size=2,
    return_dict_in_generate=True,
    output_logits=True,
    do_sample=True,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
    # eos_token_id=tokenizer.encode('\n'),
    # pad_token_id=tokenizer.encode('\n')[0],
    **config,
    )

    response_text = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
    response_text = response_text.replace(prompt, "") # response repeats the input in the begining
    response_token_ids = outputs.sequences[0].to("cpu").tolist()[len(inputs.input_ids[0]):]
    # response_embeddings = outputs.sequences[0].to("cpu").tolist()[len(inputs.input_ids[0]):]
    #response_tokens = tokenizer.convert_ids_to_tokens(response_token_ids)
    #response_logits = [l.to("cpu").tolist() for l in outputs.logits]

    return response_text

ask('Hello, how are you going?', model)

"I'm doing well, thank you. I've been busy with work and family, but I have a few minutes to catch up on things. How about you?"

In [ ]:
to_save = []
for item in data:
    question = item['model_input']
    answer = item['model_output_text']
    sample_lst = []
    for i in range(3):
        sample = ask(question, model)
        sample_lst.append(sample)

    sample_withquestion = []
    for sample in sample_lst:
        sample_withquestion.append(question + '\n' + sample)
    passage = question + '\n' + answer
    #print(sample_withquestion)

    to_eval = answer.strip() 
    doc = nlp(to_eval)
    sentences = [sent.text.strip() for sent in doc.sents] # spacy sentence tokenization
    print('sentences', sentences)
    print('passage', passage)
    print('sample_lst', sample_lst)
    sent_scores_ngram = selfcheck_ngram.predict(
        sentences = sentences,   
        passage = passage,
        sampled_passages = sample_lst,
    )
    #print(sent_scores_ngram)
    lst_logprob = sent_scores_ngram['sent_level']['lst_neg_logprob']
    lst_logprob = np.abs([item for sublist in lst_logprob for item in sublist])

    doc = nlp(to_eval)
    sentences = [sent.text.strip() for sent in doc.sents] # spacy sentence tokenization
    token_lst = []
    for sent in doc.sents:
        for token in nlp(sent.text.strip()):
            token_lst.append(token)
    print(token_lst)
    print(lst_logprob)
    df = pd.DataFrame({'tokens': token_lst, 'logprob': lst_logprob})
    to_drop = df[df['logprob'] == df['logprob'].max()]['tokens'].tolist()
    to_save.append(to_drop)


sentences ['Die spanische Provinz heißt Álava.']
passage In welcher Provinz liegt die spanische Gemeinde Cilleruelo de San Mamés?

Die spanische Provinz heißt Álava.
sample_lst ['Cillerulo is a municipality located in the province of San Sebastian, in Gipuzkoa, Spain.', 'Cillerulo is a municipality located in the province of San Sebastian, in Gipuzkoa, Spain.', 'Cillerulo de san Mamé is located in the province of San Sebastian, in northern Spain.']
test
[Die, spanische, Provinz, heißt, Álava, .]
[3.54095932 3.54095932 3.54095932 4.2341065  4.2341065  2.84781214]
sentences ['Der Film "\nBrats\n– Ein \nDorf macht  Schule" gewann den Publikumspreis beim  Internationalen  Dokumentarfilmfestival', 'München  2019.']
passage Welchen Preis gewann der Dokumentarfilm "Bratsch – Ein Dorf macht Schule"?

Der Film "
Brats
– Ein 
Dorf macht  Schule" gewann den Publikumspreis beim  Internationalen  Dokumentarfilmfestival  München  2019.
sample_lst ["Der Dokumentary-Film „Bratch - Ein dorf macht Schul

In [ ]:
print(to_save)
filtered_list = [[word for word in item if not word.is_stop] for item in to_save ]
print(filtered_list)

[[name, for, line, produced, by, Diageo, first, introduced, United, States, 1945, However, image, as, pirate, with, parrot, on, his, shoulder, did, appear, until, 1954, origin, actual, blend, well, documented, and, there, no, evidence, to, suggest, that, it, before, 's, introduction], [a, (, ), It, third, -, largest, after, Yekaterinburg, Nizhniy, known, for, its, heavy, industry, particularly, metallurgy, machine, building], [distinct, historical, although, closely, related, often, associated, each, historic, at, heart, has, main, rulers, since, 14th, various, palaces, buildings, surrounded, by, thick, walls, towers, making, formidable, on, hand, original, core, built, late, 13th, Grand, Prince, his, court, means, citadel, or, smaller, more, compact, area, within, Square, famous, such, St., Basil, 's, Archangel, Michael, So, while, both, complexes, have, residential, functions, that, one, parts], [plays, Douglas, which, is, located, South, Lanarkshire, Scotland, capacity, around, 6,00

In [38]:
from datetime import datetime 
timestamp = datetime.now().strftime("%Y%m%d%H%M%S")

token_lst_lst = []
for save in filtered_list:
    token_lst = list(token.text for token in save)
    token_lst_lst.append(token_lst)
print(token_lst_lst)

try:
    with open(f'./output/output{timestamp}.json', 'w') as f:
        json.dump(token_lst_lst, f, indent=4)
except Exception as e:
    print("发生错误:", e)

[['line', 'produced', 'Diageo', 'introduced', 'United', 'States', '1945', 'image', 'pirate', 'parrot', 'shoulder', 'appear', '1954', 'origin', 'actual', 'blend', 'documented', 'evidence', 'suggest', 'introduction'], ['(', ')', '-', 'largest', 'Yekaterinburg', 'Nizhniy', 'known', 'heavy', 'industry', 'particularly', 'metallurgy', 'machine', 'building'], ['distinct', 'historical', 'closely', 'related', 'associated', 'historic', 'heart', 'main', 'rulers', '14th', 'palaces', 'buildings', 'surrounded', 'thick', 'walls', 'towers', 'making', 'formidable', 'hand', 'original', 'core', 'built', 'late', '13th', 'Grand', 'Prince', 'court', 'means', 'citadel', 'smaller', 'compact', 'area', 'Square', 'famous', 'St.', 'Basil', 'Archangel', 'Michael', 'complexes', 'residential', 'functions', 'parts'], ['plays', 'Douglas', 'located', 'South', 'Lanarkshire', 'Scotland', 'capacity', '6,000', 'ground', '1878', 'making', 'oldest', 'grounds', 'world', 'use', 'single'], ['2004', 'formed', 'merger', 'neighbor